Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

---

### Monte-Carlo transport project
Simulate electron transport in a cylindrical Geiger counter filled 
with Helium gas.

This project attempts to simulate the microscopic transport of electrons in Helium gas under the influence of an electric field. The challenge for this modelling task is to implement the physics of electron-atom scattering with the kinematics of the Lorentz force on an electron in an electric field. Scattering is a quantum process which takes place probabilistically depending on the kinetic energy of the charge. However, this kinetic energy depends on the discrete step in time the program decides to take since the electric field keeps accelerating the electron. 

The contradiction between the requirement for a constant, discrete time step and a probabilitic waiting time step, i.e. the time to the next scattering reaction as a function of kinetic energy, is solved by the null-collision simulation method. A description of the method can be found below. Find all technical requirements for the code below.

- Write a function **run(charges, voltage)** that starts the simulation and returns (a) a list of lists of arrays of electron interaction locations in 3D in order to draw the required scatterplot, see below and (b) a list of total transport times for every electron. Input **charges** is a list of a single array with the starting position of the first electron (in 3D) and represents the charge pool container (the list) to store all electron starting locations that have not yet been transported. The **voltage** input is a number that needs to reach the electric field calculation function (for you to write) for flexibility in the simulation configuration.

- Geometry: Let the cylindrical Geiger counter long axis be defined along the z-axis. Set the anode wire with radius 8.0e-5 m at the origin; the cathode cylinder radius is 1.0e-2 m and the anode bias is set at 1000 Volt. Start a single electron at a radial distance of 3.0e-3 m.

- Transport should be simulated using the null-collision method. A pseudo-code demonstrating the method is shown and discussed below.

- Electrons can scatter elastically or ionize the gas using inelastic scattering. Each ionization should increase the number  of electrons to transport by one (store it in the charge pool). The original electron which requires further, subsequent, transport should carry on after ionization (however, see condition below). The two cross section functions have been pre-coded for you. A function randomly choosing between the processes as a function of energy in eV is also given.

- In case of elastic scattering, simply change (i.e. rotate) the velocity vector direction randomly in the forward hemisphere, i.e. towards the wire, and leave the magnitude unchanged. The only velocity magnitude (and direction) changes in the entire code should arise from acceleration due to the electric field. That 'forward' scattering statement is an arbitrary, simple, requirement. Forward scattering in electron-atom interactions is dominant but the angular distributions (which we would realistically need here) are complicated hence ignore them here.

- The case of an inelastic collision should reset the electron velocity vector to thermal energy (0.025 eV), set the dynamic null collision constant `kv=0` (see below) and store the location of the electron in the charge pool (identical to creating an additional electron since the original carries on being transported) for later transport.

- Count all interactions for all electrons and store all inelastic collision locations and 1 in 1000 elastic collision locations. Otherwise, there will be too many collisions to store and to draw! The model solution produced between 7-11 thousand electrons in test runs with the given conditions, each with thousands of interaction locations stored.

- Suppress single electron starting positions from the pool of charges closer than 5e-4 m: 

```python
if math.hypot(pos[0], pos[1]) < 5.0e-4: # block avalanche at rad=5.0e-4 m
    places.append(pos)
    stopFlag = True`
```
    
assuming the location vector 'pos' is created from the charge pool container and 'stopFlag' is the boolean checked for the transport `while` loop. This suppression should keep execution time of your pure python code below 90 minutes (on my humble 4-core machine), hopefully.

#### Marking
As always for Python projects, find the correct solution first, optimize second. A correct figure indicates a successful code hence attracts 15 out of 20 marks (with the caveat of checking the code for correctness, of course, including subtraction of marks for mistakes along the way). The final 5 marks are awarded for speed of code execution.
- Draw a scatterplot of electron transport interactions. The picture should reveal the electron avalanche near the wire for full marks on graphics.

- This simulation benefits greatly from efforts to speed up the execution. On the assessment workstation (4-cores) a speed-up factor between 20-30 has been achieved comparing the pure python version with an optimized version. Full marks will be awarded for any speed-up factor of 10 or more compared to the pure python model solution. Clearly, the given cross section functions are also available for optimization in order to reach such speed-ups. They are being called millions of times after all. Therefore, please print(!) the process time of your `run(charges, voltage)` function for a manual assessment using

```python
start = time.process_time()
res, times = run(single, bias) # just example variable names, choose as you wish
stop = time.process_time()
print('Process time: ', (stop-start))
``` 

- In case no full marks can be awarded according to the items above, partial marks will be awarded manually, depending on how many required parts of the full code have been completed.

### Null collision method and coding hints
The function transporting a single electron should contain the null collision method set up

```python
kmax = 2.0e-12 # null collision constant
tau = 1.0 / (nDensity * kmax)
```

with nDensity the number density of the gas following from the Helium density
`density = 0.1664 # [kg/m^3] NTP He gas at 293 K`
and 'tau' the time constant of the null collision method. The 'tau' constant is required in the time update function (for you to code).

`time_update = -tau * math.log(random.random())`

which yields in the single electron transport `while` loop a new `time_step=time_update` proposal. Remember to add up the individual time steps in case no collision has taken place and the electron remains in free flight.

With that proposal, the velocity vector should be updated, a new kinetic energy calculated and the corresponding cross section determined. Then the null collision decision must be take. For that calculate

`kv = np.linalg.norm(v0) * cross_section`

where cross_section results from the `cross_section(energy)` function already in the correct SI units. Note that the call to the cross section function already decides whether an inelastic collision takes place. Therefore that case can be dealt with immediately. If that is not the case, then take the (elastic) collision decision

```python
prob = random.random()
if prob <= (kv/kmax): # collision takes place
```

Deal with the collision and prepare the next step, see below, and finally return to the start of the while loop for the next step.

One crucial action to take once a collision takes place, is to advance the electron position to a new location. Change the velocity vector direction (not magnitude!) random-isotropically in the forward hemisphere (not realistic but simple) and evaluate the electric field at that place. The latter is a good place to check for a geometry boundary to stop the transport, i.e. signal the `while` to stop. That electric field vector is required in your update of the velocity, see above. 

This velocity update is an approximation (null collision means, without a collision we do not care where the electron is, just that is in free flight) since we pretend that the electron is still at the last known location while we let it fly freely. However, it is an excellent approximation and not a problem as new collisions occur in nanometre distances.

Final tip: the electric field is radially symmetric around the wire anode and points away from the wire, while the Coulomb  force on the electron points in the opposite direction. The Coulomb force determines the velocity vector to add(!) to the current one, given a time_step according to simple, non-relativistic kinematics. The constant electric charge over electron mass in SI units should be used here to retain SI units throughout (keep the electric field strength in Volt/m). The formula for the electric field calculation can be researched in most textbooks on the subject.


In [ ]:
'''
Add more import commands below as required. 
This is all that is required for the given code.
'''

from random import random, uniform
# YOUR CODE HERE
import numpy as np
import matplotlib.pyplot as plt
from math import log, sqrt, pi, sin, cos
from scipy import constants
import time
from numba import njit
from numba.typed import List

@njit(fastmath=True)
def elasticCS(energy):
    '''
    Parameters
    ----------
    energy : float
        electron energy [eV].

    Returns
    -------
    float
        elastic cross section [cm^2].

    Piece-wise cross section approximation.
    '''
    if energy <= 2.0:
        return 7.0e-16 # [cm^2]
    if energy <= 10.0:
        return energy * (-0.27231e-16) + 6.455375e-16
    if energy <= 20.0:
        return energy * (-0.20977e-16) + 6.8801e-16
    if energy <= 100.0:
        return energy * (-0.02981e-16) + 3.2809e-16
    return 3.0e-17 # [cm^2]

#constants for inelasticCS:
SLOPE = 0.047745e-17
INTERCEPT = -1.174527e-17
MAX_CROSS_SECTION = 3.6e-17

@njit(fastmath=True)
def inelasticCS(energy):
    '''
    Parameters
    ----------
    energy : float
        electron energy [eV].

    Returns
    -------
    float
        inelastic cross section [cm^2].

    Approximate the inelastic cross section as linear from
    the threshold at 24.6 eV to 100 eV, constant after that.
    Cross sections return in units of cm^2.
    '''
    return max(0.0, min(energy * SLOPE + INTERCEPT, MAX_CROSS_SECTION)) # [cm^2]

@njit(fastmath=True)
def cross_section(energy):
    '''
    Parameters
    ----------
    energy : float
        electron energy [eV].

    Returns
    -------
    inelFlag : bool
        signal inelastic scattering.
    elastic  : float
        elastic scattering cross section [m^2] or zero.

    '''
    inelFlag = False
    elastic = elasticCS(energy) * 1.0e-4 # convert to [m^2]
    inel = inelasticCS(energy) * 1.0e-4 # convert to [m^2]

    if inel <= 0.0: # only elastic
        return inelFlag, elastic
    ratio = inel / (inel + elastic)
    if random() < ratio:
        inelFlag = True
        return inelFlag, 0.0
    return inelFlag, elastic


# YOUR CODE HERE
"""
Plan for run(charges voltage)

- Charges is initially the single starting electron. 
- Start a for loop (for elec in charges:)
- remember to convert density to number density in the set up
- Start a while loop for each elec in charges which terminates when the electron gets too close to the anode
  or passes the cathode boundary (unlikely but could happen with a collision).
- remember to convert density to number density in the set up
- call time_update(tau)
- time_step += time_update
- update velocity using the acceleration caused by the electic field (a = F/m) and the timestep (have u, a, t
  and want v) / or calculate the velocity needed to be added using the Coulomb force. update the electron location.
- use velocity to calculate energy (cross_section wants it in eV)(find electron mass in 1/eV?)
- call cross_section(energy)
- if inelFlag = True, inelastic collision takes place so reset velocity vector to thermal energy but don't 
  rotate. Produce another electron with 0 velocity and store in charges. Store the location of the collision.
  Use the value of the electric field to decide if the while loop should be stopped. Set kv to 0 so elastic collision
  doesn't take place. Note that if an inelastic collision is chosen by  cross_section() then cross_section = 0.
  Therefore the kv calculation can be used no matter what the collision is.
- if inelFlag = False, calculate kv using cross_section and test if collision takes place at all (prob <= kv/kmax)
- if collision takes place, rotate velocity vector. Store the location of the collision with probability 1/1000. Use
  the value of the electric field to decide if the while loop should be stopped.
- if collision doesn't take place, while loop restarts with the electron only moved, changed energy, and flight time
  increased (all done earlier).


OTHER FUNCTIONS TO CALL:
time_update(tau)
cross_section(energy) (already built, could be optimised)

Optimisation thought:
If an inelastic collision takes place, can we avoid calculating kv completely since it's not needed?

CHECK: electric field equation

"""
#standard constants
e = constants.e #[C] elementary charge
e_m = constants.electron_mass #[kg]
eV = constants.physical_constants["electron volt"][0] #[J], Joules per electron volt
acceleration_scale_const = e/e_m #elementary charge over electron mass [C/kg]
energy_scale_const = e_m/(2*eV) #[kg/J]

@njit(fastmath=True)
def time_update(tau):
    return -tau * log(random())

@njit(fastmath=True)
def velo_update(u,a,t):
    # v = u +at
    return u + a*t

@njit(fastmath=True)
def pos_update(r,u,a,t):
    # s = ut + (at^2)/2
    return (u*t + (a*t*t)/2) + r

@njit(fastmath=True)
def acceleration(V,r):
    #electric field = voltage/(radial distance)ln(R/r) [V/m] (magnitude)
    #force = charge x electric field [N] (volt/meter = Newtons/Coulomb) (magnetude)
    #acceleration = Force / mass [m/s^2] (magnetude)
    #multiply by normalised direction vector towards the wire as acceleration goes to the wire
    radial_dist = sqrt(r[0]*r[0]+r[1]*r[1])
    E = -V/(radial_dist*log(8.0e-5/1.0e-2))
    a = E * acceleration_scale_const #ASC = e/em [C/kg]
    return a*(np.array([-r[0],-r[1],0])/radial_dist)

@njit(fastmath=True)
def rand_direc(v):

    # Random angles
    theta = uniform(0.0, pi / 2)
    phi = uniform(0.0, 2 * pi)

    # Construct local coordinate system
    # normalise v
    v = v / sqrt(v[0]*v[0]+v[1]*v[1]+v[2]*v[2])
    # u can be any normalized vector perpendicular to w
    u = np.array([-v[1], v[0], 0.0])
    if (u[0]*u[0]+u[1]*u[1]) == 0.0:  # Handle special case
        u = np.array([1.0, 0.0, 0.0])
    u = u / sqrt(u[0]*u[0]+u[1]*u[1]+u[2]*u[2])
    # w is perpendicular to both v and u
    w = np.array([v[1]*u[2] - v[2]*u[1], v[2]*u[0] - v[0]*u[2],v[0]*u[1] - v[1]*u[0]])
    # Rotate in the local coordinate system
    rotated = (sin(theta) * cos(phi) * u +
             sin(theta) * sin(phi) * w +
             cos(theta) * v)

    return rotated

@njit(fastmath=True)
def energy(mag_v):
    # E = (mv^2)/2
    return (mag_v*mag_v)*energy_scale_const #[eV] ESC = em/2eV [kg/J]

#CONSTANTS FOR run() and single()
RUN_KMAX = 2.0e-12 #null collision constant
RUN_nDENSITY = 0.1664 / (6.6464731e-27) #ndensity = density / mass per particle, mass of He = 6.6464731x10^-27 kg [particles / m^3]
RUN_TAU = 1.0 / (RUN_nDENSITY * RUN_KMAX)
RUN_TE_VELO_MAG = sqrt(2*(0.025*eV)/e_m) #from thermal energy; magnetude. Uses constants eV and e_m from scipy constants defined earlier

@njit(fastmath=True)
def single(elec_pos, voltage, charges):
    single_places = []
    init_velo_direc = np.array([-elec_pos[0],-elec_pos[1],0]) #vector from electron to wire
    init_velo = RUN_TE_VELO_MAG*(init_velo_direc/sqrt(init_velo_direc[0]*init_velo_direc[0]+init_velo_direc[1]*init_velo_direc[1])) #chosen to point the initial velocity towards the wire.
    prev_pos = elec_pos
    prev_velo = init_velo
    total_time = 0.0
    time_step = 0.0
    stopFlag = False
    while stopFlag == False:
        time_step += time_update(RUN_TAU)
        next_velo = velo_update(prev_velo,acceleration(voltage,prev_pos),time_step)
        next_velo_mag = sqrt(next_velo[0]*next_velo[0]+next_velo[1]*next_velo[1]+next_velo[2]*next_velo[2])
        inelFlag, cross_section_val = cross_section(energy(next_velo_mag))
        if inelFlag == True: #inelastic collision takes place
            next_pos = pos_update(prev_pos,prev_velo,acceleration(voltage,prev_pos),time_step)
            if next_pos[0]*next_pos[0]+next_pos[1]*next_pos[1] < 2.5e-7: #within suppression zone - (5.0e-4)^2 to avoid sqrt. For final version set to 2.5e-7
                stopFlag = True
            else:
                charges.append(next_pos)
                single_places.append(next_pos)
                total_time += time_step
                time_step = 0.0
                prev_pos = next_pos
                next_velo_direc = np.array([-next_pos[0],-next_pos[1],0]) #vector from electron to wire
                prev_velo = RUN_TE_VELO_MAG*(next_velo_direc/sqrt(next_velo_direc[0]*next_velo_direc[0]+next_velo_direc[1]*next_velo_direc[1]))#setting prev_velo for next iteration. chosen to point the velocity towards the wire
        else:
            kv = next_velo_mag * cross_section_val
            prob = random()
            if prob <= (kv/RUN_KMAX): #elastic collision takes place
                next_pos = pos_update(prev_pos,prev_velo,acceleration(voltage,prev_pos),time_step)
                if next_pos[0]*next_pos[0]+next_pos[1]*next_pos[1] < 2.5e-7: #within suppression zone - (5.0e-4)^2 to avoid sqrt. For final version set to 2.5e-7
                    stopFlag = True
                else:
                    prob2 = random()
                    if prob2 <= 0.001:
                        single_places.append(next_pos)
                    total_time += time_step
                    time_step = 0.0
                    prev_pos = next_pos
                    prev_velo = next_velo_mag*rand_direc(next_velo)
            else: #no collision at all
                #time_step not reset
                #not deciding position or velocity as the time_step is going to increase and will just calculate with that extra flight time with the s & v we had.
                pass
    return single_places, total_time

def run(charges, voltage):
    places = []
    times = []
    while len(charges)>0:
        single_places, time = single(charges.pop(), voltage, charges)
        places.append(single_places)
        times.append(time)
    return places,times


volt = 1000.0

init_elec_loc = List([np.array([3.0e-3,0.0,0.0])])

start = time.process_time()
int_loc, flight_times = run(init_elec_loc,volt)
stop = time.process_time()
print('Process time: ', (stop-start))

Note to marker:

I tried my very best to find a way to run this in parallel but it felt like I needed half a degree in computer science to be able to figure out it out since they all need to append to charges and also iterate through charges. I spent a lot of time trying this and thankfully got my process time to below 6 minutes on average just by using JIT compiling and some effective - albeit desperate - modifications to certain calculations. Unfortunately, some of these came at a cost of my code being less readable so I apologise for that.
Note also that my laptop ran this quicker when plugged in an not using battery power - I don't know if that helps or not.

In [ ]:
#Plots
def data_formatter(list_of_lists):
    x = []
    y = []
    z = []
    for elem in list_of_lists:
        for i in elem:
            x.append(i[0])
            y.append(i[1])
            z.append(i[2])
    return x,y,z

xvals,yvals,zvals = data_formatter(int_loc)

t = np.linspace(0.0,2.0*pi,100)
x_an = (8.0e-5)*np.cos(t)
y_an = (8.0e-5)*np.sin(t)

x_lim = (5.0e-4)*np.cos(t)
y_lim = (5.0e-4)*np.sin(t)

plt.figure(figsize = (6,6))
plt.xlim(-0.0035,0.0035)
plt.ylim(-0.0035,0.0035)
plt.plot(xvals,yvals,'cx',label = "Electron Interaction Locations")
plt.plot(0.003,0, 'rx', label = "Initial Electron Starting Point")
plt.plot(x_an,y_an,'y', label = "Anode")
plt.plot(x_lim,y_lim, 'k--', label = "Avalanche cut-off")
plt.xlabel("$x$ [m]")
plt.ylabel("$y$ [m]")
plt.title("Simulation results showing electron interaction locations (all inelastic, 1 in 1000 elastic)")
plt.legend()
plt.show()

plt.figure(figsize = (6,6))
plt.xlim(0.00045,0.00055)
plt.ylim(-0.0001,0.0001)
plt.plot(xvals,yvals,'cx',label = "Electron Interaction Locations")
plt.plot(x_lim,y_lim, 'k--', label = "Avalanche cut-off")
plt.xlabel("$x$ [m]")
plt.ylabel("$y$ [m]")
plt.title("Zoomed section to show the electrons stopping at the boundary (overlap due to symbols)")
plt.legend(loc = 'upper left')
plt.show()

plt.figure(figsize = (6,6))
plt.plot(xvals,yvals,'cx', label = "Electron Interaction Locations")
plt.plot(0.003,0, 'rx', label = "Initial Electron Starting Point")
plt.xlabel("$x$ [m]")
plt.ylabel("$y$ [m]")
plt.title("Electron Interaction Locations Only")
plt.legend()
plt.show()

%matplotlib qt

ax = plt.axes(projection="3d")

ax.plot(xvals,yvals,zvals, 'cx')
ax.view_init(elev=10., azim=150)
plt.show()